In [75]:
import pandas as pd
import numpy as np
import sklearn.model_selection as sk_ms
import warnings
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_excel('Airbnb.xlsx')

In [77]:
df_train, df_test = sk_ms.train_test_split(df, train_size = 0.8, random_state = 100)

In [78]:
len(df_train)

13477

# 探寻缺失情况

In [79]:
def missing_value(df_original):
   #删除'neighborhood_overview'
    df = df_original.copy()
    del df['neighborhood_overview']
    #统计每一行缺失字段的数量
    df['missing_num']=df.isnull().sum(axis=1)
    #缺失6个及以上的数据或bed=0或price=0
    len(df[(df['missing_num']>=6) | (df['beds'].isnull()) | (df['price']==0)]) 
    #剔除掉缺失6个及6个以上的
    df1=df[(df['missing_num']<6) & (df['beds'].notnull()) & (df['price']!=0)]
    df1.reset_index(inplace=True)
    del df1['index']
    #删除host_response和last review都缺失的情况
    df_new = df1[((df1['host_response_time'].notnull()) | (df1['last_review'].notnull()))]
    df_new = df_new[df_new['price']<=1000]
    df_new.reset_index(inplace=True)
    del df_new['index']
    
    #逻辑是如果last review在一年之前，使用'a few days or more'和0去填补，否则time用mode,rate用median
    df_new['host_response_time']=np.where((df_new['host_response_time'].isnull()) & (df_new['last_review']<pd.to_datetime('2021-09-07')),'a few days or more',df_new['host_response_time'])
    df_new['host_response_time']=np.where(df_new['host_response_time'].isnull() ,df_new['host_response_time'].mode(),df_new['host_response_time'])

    df_new['host_response_rate']=np.where((df_new['host_response_rate'].isnull()) & (df_new['last_review']<pd.to_datetime('2021-09-07')),float(0),df_new['host_response_rate'])
    df_new['host_response_rate']=df_new['host_response_rate'].fillna(df_new['host_response_rate'].median())
    
    #逻辑是一个用median，一个用mode
    df_new['host_acceptance_rate']=df_new['host_acceptance_rate'].fillna(df_new['host_acceptance_rate'].median())
    df_new['bathrooms_text']=np.where(df_new['bathrooms_text'].isnull(),df_new['bathrooms_text'].mode(),df_new['bathrooms_text'])
    
    
    #逻辑是根据bed分组，然后用对应bed所在组别的median填补
    bedroom_dict=dict(df_new[df_new['bedrooms'].notnull()].groupby('beds').median()['bedrooms'])
    df_new['bedrooms']=df_new.apply(lambda x:bedroom_dict[x['beds']] if not (x['bedrooms']>0) else x['bedrooms'] ,axis=1)
    
    #逻辑是last review用最早的时间，review-per-month为0，rate用median
    df_new['last_review']=df_new['last_review'].fillna(df_new['last_review'].min())
    df_new['reviews_per_month']=df_new['reviews_per_month'].fillna(0)
    df_new['review_scores_rating']=df_new['review_scores_rating'].fillna(df_new['review_scores_rating'].median())
    
    return df_new

In [80]:
df_new=missing_value(df)

In [81]:
df_new.head()

,id,name,description,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,missing_num
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",within a day,0.75,0.23,f,9.0,t,t,Midtown,Manhattan,40.75356,-73.98559,Entire home/apt,1,1 bath,1.0,1.0,"[""Stove"", ""Coffee maker"", ""Long term stays all...",175.0,30,0,49,2022-06-21,4.68,3,0.31,1
1,5178,Large Furnished Room Near B'way銆,Please don鈥檛 expect the luxury here just a bas...,within a few hours,1.00,1.00,f,1.0,t,f,Midtown,Manhattan,40.76457,-73.98317,Private room,2,1 bath,1.0,1.0,"[""Body soap"", ""Microwave"", ""Laundromat nearby""...",68.0,2,4,550,2022-08-22,4.23,1,3.39,0
2,80493,Cozy room in East Village with AC,This room is up three flights of stairs - it's...,within an hour,1.00,0.86,f,1.0,t,t,East Village,Manhattan,40.72322,-73.98615,Private room,2,1.5 shared baths,1.0,2.0,"[""Essentials"", ""Oven"", ""Cable TV"", ""Refrigerat...",55.0,2,0,207,2022-08-13,4.34,1,1.49,0
3,45936,Couldn't Be Closer To Columbia Uni,<b>The space</b><br />Hi and Thank you for che...,within a day,0.50,0.86,f,2.0,t,t,Morningside Heights,Manhattan,40.80630,-73.95985,Private room,2,1 shared bath,1.0,1.0,"[""Breakfast"", ""Cable TV"", ""Elevator"", ""Heating...",75.0,31,0,135,2022-07-11,4.65,1,0.95,0
4,5203,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",a few days or more,0.00,0.95,f,5.0,t,t,Upper West Side,Manhattan,40.80380,-73.96751,Private room,1,1 shared bath,1.0,1.0,"[""Hair dryer"", ""Essentials"", ""Breakfast"", ""Ele...",75.0,2,0,118,2017-07-21,4.91,1,0.75,3


In [82]:
percent_missing_col = df_new.isnull().sum() * 100 / len(df_new)
missing_col = pd.DataFrame({'Percent Missing': percent_missing_col})
missing_col

,Percent Missing
id,0.000000
name,0.034593
description,1.259167
host_response_time,0.000000
host_response_rate,0.000000
host_acceptance_rate,0.000000
host_is_superhost,0.000000
host_total_listings_count,0.000000
host_has_profile_pic,0.000000
host_identity_verified,0.000000


In [83]:
len(df_new)

14454

In [84]:
# Get the training dataset after filling missing values
df_train_new = df_new.reindex(df_train.index)
df_train_new.dropna(axis=0, how='all', inplace=True)
len(df_train_new)

11575

In [85]:
# Get the test dataset after filling missing values
df_test_new = df_new.reindex(df_test.index)
df_test_new.dropna(axis=0, how='all', inplace=True)
len(df_test_new)

2879

# 除填补数据外的数据预处理

In [86]:
def set_dummies(data, colname):
    for col in colname:
        data[col] = data[col].astype('category')#转换成数据类别类型，pandas用法
        dummy = pd.get_dummies(data[col])  #get_dummies为pandas里面求哑变量的包
        dummy = dummy.add_prefix('{}#'.format(col)) #add_prefix为加上前缀
        data.drop(col,axis = 1,inplace = True)
        data = data.join(dummy) 
    return data

from sklearn.preprocessing import LabelEncoder

def set_TF(data,colname):
    for col in colname:
        encoder = LabelEncoder()
        encoder.fit(data[col])
        data[col]=encoder.transform(data[col])
    return data

def bath_num(string):
    a=string.split()
    for j in a:
        try:
            float(j)
            return float(j)
        except:
            if j.lower()=='half-bath':
                return(0.5)
            continue
    return 0

def preprocessing(df_new):
    df_reg=df_new.copy()
    useless_column_list=['missing_num','name','neighbourhood_group_cleansed','latitude', 'longitude']
    for i in useless_column_list:
        try:
            del df_reg[i]
        except: 
            pass
    df_reg['last_review']=df_reg['last_review'].map(lambda x: (pd.to_datetime('2022-09-07')-pd.to_datetime(x)).days)
    ### 处理'host_response_time'
    host_response_time_dict={'within an hour':4,'within a few hours':3,'within a day':2 , 'a few days or more':1}
    df_reg['host_response_time']=df_reg['host_response_time'].apply(lambda x: host_response_time_dict[x])
    
    ### 处理''room type'
    df_reg=set_dummies(df_reg,['room_type'])
    
    
    ### 处理'host_has_profile_pic','host_identity_verified'
    df_reg=set_TF(df_reg,['host_is_superhost', 'host_has_profile_pic','host_identity_verified'])
    
    ### 处理'bathroom'
    df_reg['bathrooms_text']=df_reg['bathrooms_text'].apply(lambda x: bath_num(x))
    
    ### 处理amenities 
    amenities_list=[eval(x) for x in list(df_reg['amenities'])]
    amenities_dict={}
    for i in amenities_list:
        for j in i:
            if amenities_dict.get(j):
                amenities_dict[j]+=1
            else:
                amenities_dict[j]=1
    
    amenities_list_top=[i[0] for i in sorted(amenities_dict.items(),key=lambda x: x[1],reverse=True)[:20]]
    
    df_reg['amenities_percent']=df_reg['amenities'].apply(lambda x: len(list(set(eval(x)).intersection(set(amenities_list_top))))/len(amenities_list_top))
    
    del df_reg['amenities']
    
    #处理'neighbourhood_cleansed'
    df_reg1=df_reg.copy()
    df_reg=set_dummies(df_reg,['neighbourhood_cleansed'])
    
    return df_reg,df_reg1


In [87]:
pd.set_option('display.max_columns', None)

### Get processed training data

In [88]:
train_dummy_noscale, train_nodummy_noscale = preprocessing(df_train_new)

#### with dummy, without scale

In [89]:
train_dummy_noscale.reset_index(inplace = True, drop = True)
train_dummy_noscale.head()

,id,description,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,amenities_percent,neighbourhood_cleansed#Battery Park City,neighbourhood_cleansed#Central Park,neighbourhood_cleansed#Chelsea,neighbourhood_cleansed#Chinatown,neighbourhood_cleansed#Civic Center,neighbourhood_cleansed#East Harlem,neighbourhood_cleansed#East Village,neighbourhood_cleansed#Ellis Island,neighbourhood_cleansed#Financial District,neighbourhood_cleansed#Flatiron District,neighbourhood_cleansed#Gramercy,neighbourhood_cleansed#Greenwich Village,neighbourhood_cleansed#Harlem,neighbourhood_cleansed#Hell's Kitchen,neighbourhood_cleansed#Inwood,neighbourhood_cleansed#Kips Bay,neighbourhood_cleansed#Little Italy,neighbourhood_cleansed#Lower East Side,neighbourhood_cleansed#Marble Hill,neighbourhood_cleansed#Midtown,neighbourhood_cleansed#Morningside Heights,neighbourhood_cleansed#Murray Hill,neighbourhood_cleansed#NoHo,neighbourhood_cleansed#Nolita,neighbourhood_cleansed#Roosevelt Island,neighbourhood_cleansed#SoHo,neighbourhood_cleansed#Stuyvesant Town,neighbourhood_cleansed#Theater District,neighbourhood_cleansed#Tribeca,neighbourhood_cleansed#Two Bridges,neighbourhood_cleansed#Upper East Side,neighbourhood_cleansed#Upper West Side,neighbourhood_cleansed#Washington Heights,neighbourhood_cleansed#West Village
0,4.411786e+07,A truly epic loft in the heart of Greenwich vi...,1,0.00,0.57,0,6.0,1,1,2.0,1.0,1.0,1.0,499.0,5.0,56.0,1.0,122,5.00,1.0,0.24,1,0,0,0,0.55,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,6.592306e+17,A spacious unit which sleeps eight people on ...,4,1.00,1.00,0,55.0,1,1,8.0,1.0,1.0,4.0,223.0,1.0,35.0,2.0,16,5.00,51.0,1.36,0,0,1,0,0.70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.278934e+07,Charming one bedroom apartment in the heart of...,1,0.00,0.95,0,1.0,1,1,2.0,1.0,1.0,1.0,170.0,2.0,0.0,13.0,1059,4.92,1.0,0.31,1,0,0,0,0.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4.842584e+07,The apartment is located in the charming neigh...,4,1.00,1.00,1,4.0,1,1,3.0,1.0,1.0,1.0,145.0,3.0,15.0,45.0,5,4.80,4.0,2.54,1,0,0,0,0.85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6.543521e+17,Centrally located on Central Park South this s...,2,0.79,0.71,0,2.0,1,1,2.0,1.0,1.0,1.0,450.0,1.0,55.0,3.0,10,5.00,1.0,1.17,1,0,0,0,0.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### without dummy, without scale

In [90]:
train_nodummy_noscale.reset_index(inplace = True, drop = True)
train_nodummy_noscale.head()

,id,description,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,amenities_percent
0,4.411786e+07,A truly epic loft in the heart of Greenwich vi...,1,0.00,0.57,0,6.0,1,1,Greenwich Village,2.0,1.0,1.0,1.0,499.0,5.0,56.0,1.0,122,5.00,1.0,0.24,1,0,0,0,0.55
1,6.592306e+17,A spacious unit which sleeps eight people on ...,4,1.00,1.00,0,55.0,1,1,Midtown,8.0,1.0,1.0,4.0,223.0,1.0,35.0,2.0,16,5.00,51.0,1.36,0,0,1,0,0.70
2,3.278934e+07,Charming one bedroom apartment in the heart of...,1,0.00,0.95,0,1.0,1,1,West Village,2.0,1.0,1.0,1.0,170.0,2.0,0.0,13.0,1059,4.92,1.0,0.31,1,0,0,0,0.75
3,4.842584e+07,The apartment is located in the charming neigh...,4,1.00,1.00,1,4.0,1,1,Inwood,3.0,1.0,1.0,1.0,145.0,3.0,15.0,45.0,5,4.80,4.0,2.54,1,0,0,0,0.85
4,6.543521e+17,Centrally located on Central Park South this s...,2,0.79,0.71,0,2.0,1,1,Midtown,2.0,1.0,1.0,1.0,450.0,1.0,55.0,3.0,10,5.00,1.0,1.17,1,0,0,0,0.25


#### with dummy, with scale

In [91]:
from sklearn.preprocessing import StandardScaler

In [92]:
need_scale = train_dummy_noscale.iloc[:,[2,3,4,6,9,10,11,12,14,15,16,17,18,19,20,25]]
noneed_scale = train_dummy_noscale.drop(train_dummy_noscale.columns[[2,3,4,6,9,10,11,12,13,14,15,16,17,18,19,20,25]],axis = 1)

In [93]:
scaler1 = StandardScaler()
# Fit on columns that are not dummy and not price
scaler1.fit(need_scale)
need_scale = scaler1.transform(need_scale)
need_scale = pd.DataFrame(need_scale, columns=train_dummy_noscale.columns[[2,3,4,6,9,10,11,12,14,15,16,17,18,19,20,25]])

In [94]:
train_dummy_scale = pd.concat(
    [noneed_scale, need_scale,train_dummy_noscale[['price']]],axis = 1)
train_dummy_scale.head()

,id,description,host_is_superhost,host_has_profile_pic,host_identity_verified,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,neighbourhood_cleansed#Battery Park City,neighbourhood_cleansed#Central Park,neighbourhood_cleansed#Chelsea,neighbourhood_cleansed#Chinatown,neighbourhood_cleansed#Civic Center,neighbourhood_cleansed#East Harlem,neighbourhood_cleansed#East Village,neighbourhood_cleansed#Ellis Island,neighbourhood_cleansed#Financial District,neighbourhood_cleansed#Flatiron District,neighbourhood_cleansed#Gramercy,neighbourhood_cleansed#Greenwich Village,neighbourhood_cleansed#Harlem,neighbourhood_cleansed#Hell's Kitchen,neighbourhood_cleansed#Inwood,neighbourhood_cleansed#Kips Bay,neighbourhood_cleansed#Little Italy,neighbourhood_cleansed#Lower East Side,neighbourhood_cleansed#Marble Hill,neighbourhood_cleansed#Midtown,neighbourhood_cleansed#Morningside Heights,neighbourhood_cleansed#Murray Hill,neighbourhood_cleansed#NoHo,neighbourhood_cleansed#Nolita,neighbourhood_cleansed#Roosevelt Island,neighbourhood_cleansed#SoHo,neighbourhood_cleansed#Stuyvesant Town,neighbourhood_cleansed#Theater District,neighbourhood_cleansed#Tribeca,neighbourhood_cleansed#Two Bridges,neighbourhood_cleansed#Upper East Side,neighbourhood_cleansed#Upper West Side,neighbourhood_cleansed#Washington Heights,neighbourhood_cleansed#West Village,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,amenities_percent,price
0,4.411786e+07,A truly epic loft in the heart of Greenwich vi...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.386961,-1.583685,-1.328821,-0.225201,-0.521051,-0.318656,-0.445373,-0.593900,-0.436162,2.207695,-0.430537,-0.714281,0.520142,-0.345682,-0.309587,-0.679270,499.0
1,6.592306e+17,A spacious unit which sleeps eight people on ...,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.931763,0.727974,0.637967,-0.167823,3.092590,-0.318656,-0.445373,2.628974,-0.548005,1.102128,-0.413495,-0.790039,0.520142,0.264057,0.203000,0.044344,223.0
2,3.278934e+07,Charming one bedroom apartment in the heart of...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1.386961,-1.583685,0.409270,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.520044,-0.740483,-0.226034,-0.044613,0.409726,-0.345682,-0.277550,0.285549,170.0
3,4.842584e+07,The apartment is located in the charming neigh...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.931763,0.727974,0.637967,-0.227543,0.081222,-0.318656,-0.445373,-0.593900,-0.492083,0.049208,0.319305,-0.797900,0.244102,-0.309097,0.743046,0.767958,145.0
4,6.543521e+17,Centrally located on Central Park South this s...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.614053,0.242525,-0.688471,-0.229885,-0.521051,-0.318656,-0.445373,-0.593900,-0.548005,2.155049,-0.396453,-0.794327,0.520142,-0.345682,0.116043,-2.126497,450.0


In [95]:
train_dummy_scale.to_excel('train_dummy.xlsx')

#### without dummy, with scale

In [96]:
need_scale = train_nodummy_noscale.iloc[:,[2,3,4,6,10,11,12,13,15,16,17,18,19,20,21,26]]
noneed_scale = train_nodummy_noscale.drop(train_nodummy_noscale.columns[[2,3,4,6,10,11,12,13,14,15,16,17,18,19,20,21,26]],axis = 1)

In [97]:
scaler2 = StandardScaler()
# Fit on columns that are not dummy and not price
scaler2.fit(need_scale)
need_scale = scaler2.transform(need_scale)
need_scale = pd.DataFrame(need_scale, columns=train_nodummy_noscale.columns[[2,3,4,6,10,11,12,13,15,16,17,18,19,20,21,26]])

In [98]:
train_nodummy_scale = pd.concat(
    [noneed_scale, need_scale,train_nodummy_noscale[['price']]],axis = 1)
train_nodummy_scale.head()

,id,description,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,amenities_percent,price
0,4.411786e+07,A truly epic loft in the heart of Greenwich vi...,0,1,1,Greenwich Village,1,0,0,0,-1.386961,-1.583685,-1.328821,-0.225201,-0.521051,-0.318656,-0.445373,-0.593900,-0.436162,2.207695,-0.430537,-0.714281,0.520142,-0.345682,-0.309587,-0.679270,499.0
1,6.592306e+17,A spacious unit which sleeps eight people on ...,0,1,1,Midtown,0,0,1,0,0.931763,0.727974,0.637967,-0.167823,3.092590,-0.318656,-0.445373,2.628974,-0.548005,1.102128,-0.413495,-0.790039,0.520142,0.264057,0.203000,0.044344,223.0
2,3.278934e+07,Charming one bedroom apartment in the heart of...,0,1,1,West Village,1,0,0,0,-1.386961,-1.583685,0.409270,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.520044,-0.740483,-0.226034,-0.044613,0.409726,-0.345682,-0.277550,0.285549,170.0
3,4.842584e+07,The apartment is located in the charming neigh...,1,1,1,Inwood,1,0,0,0,0.931763,0.727974,0.637967,-0.227543,0.081222,-0.318656,-0.445373,-0.593900,-0.492083,0.049208,0.319305,-0.797900,0.244102,-0.309097,0.743046,0.767958,145.0
4,6.543521e+17,Centrally located on Central Park South this s...,0,1,1,Midtown,1,0,0,0,-0.614053,0.242525,-0.688471,-0.229885,-0.521051,-0.318656,-0.445373,-0.593900,-0.548005,2.155049,-0.396453,-0.794327,0.520142,-0.345682,0.116043,-2.126497,450.0


In [99]:
train_nodummy_scale.to_excel('train_nodummy.xlsx')

### Get processed test data

In [100]:
test_dummy_noscale, test_nodummy_noscale = preprocessing(df_test_new)

#### with dummy, without scale

In [101]:
test_dummy_noscale.reset_index(inplace = True, drop = True)
test_dummy_noscale.head()

,id,description,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,amenities_percent,neighbourhood_cleansed#Battery Park City,neighbourhood_cleansed#Central Park,neighbourhood_cleansed#Chelsea,neighbourhood_cleansed#Chinatown,neighbourhood_cleansed#Civic Center,neighbourhood_cleansed#East Harlem,neighbourhood_cleansed#East Village,neighbourhood_cleansed#Ellis Island,neighbourhood_cleansed#Financial District,neighbourhood_cleansed#Flatiron District,neighbourhood_cleansed#Gramercy,neighbourhood_cleansed#Greenwich Village,neighbourhood_cleansed#Harlem,neighbourhood_cleansed#Hell's Kitchen,neighbourhood_cleansed#Inwood,neighbourhood_cleansed#Kips Bay,neighbourhood_cleansed#Little Italy,neighbourhood_cleansed#Lower East Side,neighbourhood_cleansed#Marble Hill,neighbourhood_cleansed#Midtown,neighbourhood_cleansed#Morningside Heights,neighbourhood_cleansed#Murray Hill,neighbourhood_cleansed#NoHo,neighbourhood_cleansed#Nolita,neighbourhood_cleansed#Roosevelt Island,neighbourhood_cleansed#SoHo,neighbourhood_cleansed#Stuyvesant Town,neighbourhood_cleansed#Theater District,neighbourhood_cleansed#Tribeca,neighbourhood_cleansed#Two Bridges,neighbourhood_cleansed#Upper East Side,neighbourhood_cleansed#Upper West Side,neighbourhood_cleansed#Washington Heights,neighbourhood_cleansed#West Village
0,5.178000e+03,Please don鈥檛 expect the luxury here just a bas...,3,1.00,1.00,0,1.0,1,0,2.0,1.0,1.0,1.0,68.0,2.0,4.0,550.0,16,4.23,1.0,3.39,0,0,1,0,0.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5.933117e+17,This cubicle room is just for you and nobody e...,4,1.00,1.00,0,25.0,1,1,1.0,1.0,1.0,1.0,92.0,1.0,32.0,25.0,10,4.36,22.0,5.07,0,0,1,0,0.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4.726987e+07,The best Central location. <br />Union Square ...,2,0.60,0.36,0,1.0,1,1,2.0,1.0,1.0,1.0,414.0,7.0,39.0,2.0,18,5.00,1.0,0.11,1,0,0,0,0.6,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4.509560e+06,BRAND NEW !!! Fully furnished Studio Apartment...,3,0.91,0.75,0,48.0,1,1,3.0,1.0,1.0,2.0,130.0,30.0,0.0,18.0,199,4.35,22.0,0.20,1,0,0,0,0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1.118031e+06,Private Bedroom for one in the best neighborho...,2,1.00,0.67,1,3.0,1,0,1.0,1.0,1.0,1.0,136.0,21.0,27.0,55.0,56,4.94,3.0,0.50,0,0,1,0,0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### without dummy, without scale

In [102]:
test_nodummy_noscale.reset_index(inplace = True, drop = True)
test_nodummy_noscale.head()

,id,description,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,amenities_percent
0,5.178000e+03,Please don鈥檛 expect the luxury here just a bas...,3,1.00,1.00,0,1.0,1,0,Midtown,2.0,1.0,1.0,1.0,68.0,2.0,4.0,550.0,16,4.23,1.0,3.39,0,0,1,0,0.7
1,5.933117e+17,This cubicle room is just for you and nobody e...,4,1.00,1.00,0,25.0,1,1,Chelsea,1.0,1.0,1.0,1.0,92.0,1.0,32.0,25.0,10,4.36,22.0,5.07,0,0,1,0,0.7
2,4.726987e+07,The best Central location. <br />Union Square ...,2,0.60,0.36,0,1.0,1,1,Gramercy,2.0,1.0,1.0,1.0,414.0,7.0,39.0,2.0,18,5.00,1.0,0.11,1,0,0,0,0.6
3,4.509560e+06,BRAND NEW !!! Fully furnished Studio Apartment...,3,0.91,0.75,0,48.0,1,1,Stuyvesant Town,3.0,1.0,1.0,2.0,130.0,30.0,0.0,18.0,199,4.35,22.0,0.20,1,0,0,0,0.9
4,1.118031e+06,Private Bedroom for one in the best neighborho...,2,1.00,0.67,1,3.0,1,0,Nolita,1.0,1.0,1.0,1.0,136.0,21.0,27.0,55.0,56,4.94,3.0,0.50,0,0,1,0,0.9


#### with dummy, with scale

In [103]:
need_scale = test_dummy_noscale.iloc[:,[2,3,4,6,9,10,11,12,14,15,16,17,18,19,20,25]]
noneed_scale = test_dummy_noscale.drop(train_dummy_noscale.columns[[2,3,4,6,9,10,11,12,13,14,15,16,17,18,19,20,25]],axis = 1)

In [104]:
need_scale = scaler1.transform(need_scale)
need_scale = pd.DataFrame(need_scale, columns=test_dummy_noscale.columns[[2,3,4,6,9,10,11,12,14,15,16,17,18,19,20,25]])

In [105]:
test_dummy_scale = pd.concat(
    [noneed_scale, need_scale,test_dummy_noscale[['price']]],axis = 1)
test_dummy_scale.head()

,id,description,host_is_superhost,host_has_profile_pic,host_identity_verified,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,neighbourhood_cleansed#Battery Park City,neighbourhood_cleansed#Central Park,neighbourhood_cleansed#Chelsea,neighbourhood_cleansed#Chinatown,neighbourhood_cleansed#Civic Center,neighbourhood_cleansed#East Harlem,neighbourhood_cleansed#East Village,neighbourhood_cleansed#Ellis Island,neighbourhood_cleansed#Financial District,neighbourhood_cleansed#Flatiron District,neighbourhood_cleansed#Gramercy,neighbourhood_cleansed#Greenwich Village,neighbourhood_cleansed#Harlem,neighbourhood_cleansed#Hell's Kitchen,neighbourhood_cleansed#Inwood,neighbourhood_cleansed#Kips Bay,neighbourhood_cleansed#Little Italy,neighbourhood_cleansed#Lower East Side,neighbourhood_cleansed#Marble Hill,neighbourhood_cleansed#Midtown,neighbourhood_cleansed#Morningside Heights,neighbourhood_cleansed#Murray Hill,neighbourhood_cleansed#NoHo,neighbourhood_cleansed#Nolita,neighbourhood_cleansed#Roosevelt Island,neighbourhood_cleansed#SoHo,neighbourhood_cleansed#Stuyvesant Town,neighbourhood_cleansed#Theater District,neighbourhood_cleansed#Tribeca,neighbourhood_cleansed#Two Bridges,neighbourhood_cleansed#Upper East Side,neighbourhood_cleansed#Upper West Side,neighbourhood_cleansed#Washington Heights,neighbourhood_cleansed#West Village,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,amenities_percent,price
0,5.178000e+03,Please don鈥檛 expect the luxury here just a bas...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.158855,0.727974,0.637967,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.520044,-0.529899,8.925436,-0.790039,-0.542610,-0.345682,1.132063,0.044344,68.0
1,5.933117e+17,This cubicle room is just for you and nobody e...,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.931763,0.727974,0.637967,-0.202952,-1.123325,-0.318656,-0.445373,-0.593900,-0.548005,0.944190,-0.021532,-0.794327,-0.363184,-0.089591,1.900943,0.044344,92.0
2,4.726987e+07,The best Central location. <br />Union Square ...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.614053,-0.196690,-2.289345,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.380241,1.312712,-0.413495,-0.788609,0.520142,-0.345682,-0.369083,-0.438065,414.0
3,4.509560e+06,BRAND NEW !!! Fully furnished Studio Apartment...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.158855,0.519924,-0.505514,-0.176020,0.081222,-0.318656,-0.445373,0.480392,0.262854,-0.740483,-0.140825,-0.659250,-0.376986,-0.089591,-0.327893,1.009162,130.0
4,1.118031e+06,Private Bedroom for one in the best neighborho...,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-0.614053,0.727974,-0.871428,-0.228714,-1.123325,-0.318656,-0.445373,-0.593900,0.011208,0.680960,0.489723,-0.761451,0.437330,-0.321292,-0.190593,1.009162,136.0


In [106]:
test_dummy_scale.to_excel('test_dummy.xlsx')

#### without dummy, with scale

In [107]:
need_scale = test_nodummy_noscale.iloc[:,[2,3,4,6,10,11,12,13,15,16,17,18,19,20,21,26]]
noneed_scale = test_nodummy_noscale.drop(test_nodummy_noscale.columns[[2,3,4,6,10,11,12,13,14,15,16,17,18,19,20,21,26]],axis = 1)

In [108]:
need_scale = scaler2.transform(need_scale)
need_scale = pd.DataFrame(need_scale, columns=test_nodummy_noscale.columns[[2,3,4,6,10,11,12,13,15,16,17,18,19,20,21,26]])

In [109]:
test_nodummy_scale = pd.concat(
    [noneed_scale, need_scale,test_nodummy_noscale[['price']]],axis = 1)
test_nodummy_scale.head()

,id,description,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,room_type#Entire home/apt,room_type#Hotel room,room_type#Private room,room_type#Shared room,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,availability_60,number_of_reviews,last_review,review_scores_rating,calculated_host_listings_count,reviews_per_month,amenities_percent,price
0,5.178000e+03,Please don鈥檛 expect the luxury here just a bas...,0,1,0,Midtown,0,0,1,0,0.158855,0.727974,0.637967,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.520044,-0.529899,8.925436,-0.790039,-0.542610,-0.345682,1.132063,0.044344,68.0
1,5.933117e+17,This cubicle room is just for you and nobody e...,0,1,1,Chelsea,0,0,1,0,0.931763,0.727974,0.637967,-0.202952,-1.123325,-0.318656,-0.445373,-0.593900,-0.548005,0.944190,-0.021532,-0.794327,-0.363184,-0.089591,1.900943,0.044344,92.0
2,4.726987e+07,The best Central location. <br />Union Square ...,0,1,1,Gramercy,1,0,0,0,-0.614053,-0.196690,-2.289345,-0.231056,-0.521051,-0.318656,-0.445373,-0.593900,-0.380241,1.312712,-0.413495,-0.788609,0.520142,-0.345682,-0.369083,-0.438065,414.0
3,4.509560e+06,BRAND NEW !!! Fully furnished Studio Apartment...,0,1,1,Stuyvesant Town,1,0,0,0,0.158855,0.519924,-0.505514,-0.176020,0.081222,-0.318656,-0.445373,0.480392,0.262854,-0.740483,-0.140825,-0.659250,-0.376986,-0.089591,-0.327893,1.009162,130.0
4,1.118031e+06,Private Bedroom for one in the best neighborho...,1,1,0,Nolita,0,0,1,0,-0.614053,0.727974,-0.871428,-0.228714,-1.123325,-0.318656,-0.445373,-0.593900,0.011208,0.680960,0.489723,-0.761451,0.437330,-0.321292,-0.190593,1.009162,136.0


In [110]:
test_nodummy_scale.to_excel('test_nodummy.xlsx')